In [329]:
import numpy as np
import sys
import os
from scipy.spatial.transform import Rotation

# add the libfranka library, insert the franka_interface path, then import
sys.path.insert(0, "/home/luke/libs/franka/franka_interface/build")
import pyfranka_interface

# create franka controller instance
franka = pyfranka_interface.Robot_("172.16.0.2", False, False)

# create identity matrix (must be floats!)
T = np.array(
  [[1,0,0,0],
  [0,1,0,0],
  [0,0,1,0],
  [0,0,0,1]],
  dtype=np.float32
)

# create up and down matrices
move = 2e-3
up = np.copy(T)
down = np.copy(T)
up[2,3] = move
down[2, 3] = -move

# define duration in seconds
duration = 0.75

In [3]:
Z_POS = 0e-3

In [110]:
move = 10e-3
up[2,3] = move
franka.move("relative", up, duration * 2)
Z_POS += move

In [89]:
move = 2e-3
down[2, 3] = -move
franka.move("relative", down, duration)
Z_POS -= move

In [4]:
state = franka.getState()

vec_str = "[{0:.8f}, {1:.8f}, {2:.8f}, {3:.8f}, {4:.8f}, {5:.8f}, {6:.8f}]".format(*state.q)

print("cal_X_mm =", vec_str)
print(f"cal_{Z_POS * 1000:.0f}_mm = {vec_str}")

cal_X_mm = [-0.34987991, -0.01185062, 0.04871605, -0.89401940, 0.07516820, 0.92781078, 1.32547265]
cal_0_mm = [-0.34987991, -0.01185062, 0.04871605, -0.89401940, 0.07516820, 0.92781078, 1.32547265]
[[-0.00804758 -0.99817417  0.0598629   0.39983751]
 [-0.99898582  0.0106774   0.0437415  -0.11299853]
 [-0.04430082 -0.05945018 -0.99724777  0.85510062]
 [ 0.          0.          0.          1.        ]]


In [350]:
t = franka.getTransformationMatrix()
p = franka.position()
r = franka.orientation()
print("Transformation matrix is:\n", t)
print("Position is:", p)
print("Rotation is:", r)

Transformation matrix is:
 [[ 9.58184055e-01 -2.86152611e-01  1.26317999e-05  4.76389015e-01]
 [-2.86152611e-01 -9.58184055e-01 -6.48834740e-07  3.14311187e-01]
 [ 1.22892550e-05 -2.99291942e-06 -1.00000000e+00  5.27288983e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Position is: [0.47638902 0.31431119 0.52728898]
Rotation is: [3.14159200e+00 1.26317999e-05 2.90209124e-01]


In [348]:
# set the franka to be vertical, without changing other rotation angles
t = franka.getTransformationMatrix()
straighten = np.copy(t)

# set z axis to vertically downwards
straighten[2, :3] = 0
straighten[:3, 2] = 0
straighten[2, 2] = -1

print("Old transformation matrix\n", np.round(t, 2))
print("New transformation matrix\n", np.round(straighten, 2))

Old transformation matrix
 [[ 0.96 -0.29  0.02  0.48]
 [-0.29 -0.96  0.02  0.31]
 [ 0.01 -0.02 -1.    0.53]
 [ 0.    0.    0.    1.  ]]
New transformation matrix
 [[ 0.96 -0.29  0.    0.48]
 [-0.29 -0.96  0.    0.31]
 [ 0.    0.   -1.    0.53]
 [ 0.    0.    0.    1.  ]]


In [349]:
# RISKY: move into the new 'straightened' position
if True: franka.move("absolute", straighten, 5)

In [304]:
# set desired z rotation
z_angle_deg = 135 # 135 is neutral position. 75 min, 195 max

# current code is repeatable from 90-180 ONLY due to sign checks

# now construction the rotation matrix with vertical euler angles
r = franka.orientation()
t = franka.getTransformationMatrix()
rotated = np.copy(t)
print("Franka rotated euler angles", np.round(r, 3))
print("Current z rotation degrees:", np.round(r[2] * (180/np.pi), 2))

z_rot = np.arctan2(t[1, 0], t[0, 0])
print("existing z_rot degrees is:", np.round(z_rot * (180/np.pi)))

# apply the new z rotation
rotated[0,0] = np.cos(z_angle_deg * (np.pi/180))
rotated[0,1] = np.sin(z_angle_deg * (np.pi/180))
rotated[1,0] = np.sin(z_angle_deg * (np.pi/180))
rotated[1,1] = -np.cos(z_angle_deg * (np.pi/180))

# print("Old transformation matrix\n", np.round(t, 2))
# print("New transformation matrix\n", np.round(rotated, 2))

r[2] = z_angle_deg * (np.pi/180)
r_new = Rotation.from_euler("xyz", r).as_matrix()

tol = 1e-5
if r_new[1, 0] > 0 and z_angle_deg < 180 - tol:
  r_new[:2, :2] *= -1
  print("flipped")
elif abs(z_angle_deg - 180) < tol:
  if r_new[0, 0] < 0:
    r_new[:2, :2] *= -1
    print("flipped 180")

tol = 1e-5
if rotated[1, 0] > 0 and z_angle_deg < 180 + tol:
  rotated[:2, :2] *= -1
  print("flipped")
elif z_angle_deg > 180 - tol:
  if rotated[0, 0] < 0:
    rotated[:2, :2] *= -1
    print("flipped 180")

# # now assemble into the transformation matrix
# rotated[:3, :3] = r_new

print("Old transformation matrix\n", np.round(t, 2))
print("rnew rotation matrix\n", np.round(r_new, 2))
print("New transformation matrix\n", np.round(rotated, 2))

Franka rotated euler angles [ 3.14e+00 -2.00e-03 -2.62e-01]
Current z rotation degrees: -15.0
existing z_rot degrees is: 15.0
flipped
flipped
Old transformation matrix
 [[ 0.97  0.26 -0.    0.46]
 [ 0.26 -0.97 -0.    0.03]
 [-0.    0.   -1.    0.74]
 [ 0.    0.    0.    1.  ]]
rnew rotation matrix
 [[ 0.71 -0.71 -0.  ]
 [-0.71 -0.71  0.  ]
 [ 0.    0.   -1.  ]]
New transformation matrix
 [[ 0.71 -0.71 -0.    0.46]
 [-0.71 -0.71 -0.    0.03]
 [-0.    0.   -1.    0.74]
 [ 0.    0.    0.    1.  ]]


In [305]:
# RISKY: move into the new 'straightened' position
if True: franka.move("absolute", rotated, 10)

In [317]:
def set_panda_cartesian(x, y, z=0.75, zrotdeg=0, duration=10):
  """
  Send panda to particular xyz position (ABSOLUTE CO-ORDINATES). 

  Orientation locked to vertical, with zrotation being possible to set
  """

  # cartesian position limits
  xmin = 0.2
  xmax = 0.8
  ymin = -0.2
  ymax = 0.2
  zmin = 0.5
  zmax = 0.8
  zrotmin = 60
  zrotmax = -60
  zrotoffset = 135

  # apply offset for simplicity to zrot
  zrotdeg += zrotoffset

  # constrain zrotdeg to be in the range 
  while zrotdeg > zrotmax: zrotdeg -= 120
  while zrotdeg < zrotmin: zrotdeg += 120

  # constrain z height to sensible range
  if x < xmin: x = xmin
  if x > xmax: x = xmax

  # constrain z height to sensible range
  if y < ymin: y = ymin
  if y > ymax: y = ymax

  # constrain z height to sensible range
  if z < zmin: z = zmin
  if z > zmax: z = zmax

  # create identity matrix (must be floats!)
  transform = np.array(
    [[1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1]],
    dtype=np.float32
  )

  # apply the xyz positions in absolute space
  transform[0, 3] = x
  transform[1, 3] = y
  transform[2, 3] = z

  # set orientation to vertically downwards
  transform[2, 2] = -1

  # set orientation to zrot specified
  transform[0,0] = np.cos(zrotdeg * (np.pi/180))
  transform[0,1] = np.sin(zrotdeg * (np.pi/180))
  transform[1,0] = np.sin(zrotdeg * (np.pi/180))
  transform[1,1] = -np.cos(zrotdeg * (np.pi/180))

  # ensure signs are correct, constraint possible angles
  tol = 1e-5
  if transform[1, 0] > 0 and zrotdeg < 180 + tol:
    transform[:2, :2] *= -1
  elif zrotdeg > 180 - tol:
    if transform[0, 0] < 0:
      transform[:2, :2] *= -1
  if zrotdeg < 90 or zrotdeg > 195:
    print(f"zrotdeg = {zrotdeg} outside 90-195 safe limits. Aborting")
    return False
  
  print(f"Setting panda to cartesian xyz = ({x:.3f}, {y:.3f}, {z:.3f}) and zrot = {zrotdeg} deg")
  print(f"Transformation matrix is:\n{transform}")
  
  # move the panda
  franka.move("absolute", transform, duration)

  print("Panda set to the target cartesian state")

  return True

In [321]:
set_panda_cartesian(x=0.45, y=0.15, z=0.75, zrotdeg=0)

Setting panda to cartesian xyz = (0.450, 0.150, 0.750) and zrot = 135 deg
Transformation matrix is:
[[ 0.70710677 -0.70710677  0.          0.45      ]
 [-0.70710677 -0.70710677  0.          0.15      ]
 [ 0.          0.         -1.          0.75      ]
 [ 0.          0.          0.          1.        ]]
Panda set to the target cartesian state


True

In [83]:
cal = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 40, 50, 60, 70, 80, 90, 100]
# cal = [start + 2*x for x in range(25)]

start = cal[0]
end = cal[-1]

from time import sleep

Z_POS = start * 1e-3

for i in range(len(cal)):

  state = franka.getState()
  vec_str = "[{0:.8f}, {1:.8f}, {2:.8f}, {3:.8f}, {4:.8f}, {5:.8f}, {6:.8f}]".format(*state.q)
  print(f"cal_{Z_POS * 1000:.0f}_mm = {vec_str}")

  if i == len(cal) - 1: break
  if cal[i + 1] > end: break

  move = (cal[i + 1] - cal[i]) * 1e-3
  up[2,3] = move
  franka.move("relative", up, duration * 2)
  Z_POS += move

  sleep(0.3)

cal_0_mm = [-0.22735047, 0.07764017, 0.29187174, -1.80121015, -0.02719755, 1.87792673, 0.89952540]
cal_2_mm = [-0.22735047, 0.07766074, 0.29186202, -1.79806534, -0.02720234, 1.87358750, 0.89929920]
cal_4_mm = [-0.22735409, 0.07770124, 0.29185190, -1.79434772, -0.02718927, 1.86849952, 0.89923834]
cal_6_mm = [-0.22735242, 0.07762914, 0.29185190, -1.78994878, -0.02716141, 1.86361421, 0.89911325]
cal_8_mm = [-0.22735284, 0.07700316, 0.29185283, -1.78530338, -0.02711940, 1.85858461, 0.89897425]
cal_10_mm = [-0.22734769, 0.07652020, 0.29185035, -1.78081513, -0.02667632, 1.85358087, 0.89886547]
cal_12_mm = [-0.22734449, 0.07607479, 0.29184860, -1.77634064, -0.02661313, 1.84858942, 0.89873454]
cal_14_mm = [-0.22734173, 0.07565648, 0.29185035, -1.77169450, -0.02656403, 1.84356910, 0.89861984]
cal_16_mm = [-0.22733150, 0.07527275, 0.29185397, -1.76711536, -0.02651769, 1.83852584, 0.89852024]
cal_18_mm = [-0.22732211, 0.07492007, 0.29185397, -1.76240957, -0.02648662, 1.83359266, 0.89842931]
cal_2

In [326]:
# new calibrations 75DEGREE FINGERS BARE TABLE (NO WRIST) 4mm=light touch, 2mm=press
cal_0_mm = [-0.27387776, -0.02585006, 0.30729622, -1.89643083, 0.00676038, 1.87621824, 0.84037356]
cal_2_mm = [-0.27392039, -0.02572424, 0.30729622, -1.89337982, 0.00672408, 1.87142800, 0.84036733]
cal_4_mm = [-0.27400394, -0.02574353, 0.30729093, -1.88897988, 0.00673235, 1.86650059, 0.84031136]
cal_6_mm = [-0.27421166, -0.02623554, 0.30729463, -1.88446972, 0.00674949, 1.86159163, 0.84021751]
cal_8_mm = [-0.27424804, -0.02660902, 0.30729630, -1.88004050, 0.00677693, 1.85652403, 0.84012095]
cal_10_mm = [-0.27428154, -0.02698982, 0.30729559, -1.87545015, 0.00681192, 1.85143192, 0.84002916]
cal_12_mm = [-0.27454526, -0.02738393, 0.30730011, -1.87072350, 0.00689858, 1.84639242, 0.83994788]
cal_14_mm = [-0.27458083, -0.02776590, 0.30729819, -1.86601039, 0.00700295, 1.84137630, 0.83987414]
cal_16_mm = [-0.27461111, -0.02810298, 0.30729511, -1.86128947, 0.00710709, 1.83632393, 0.83981037]
cal_18_mm = [-0.27463627, -0.02841951, 0.30729817, -1.85654005, 0.00718308, 1.83133528, 0.83975061]
cal_20_mm = [-0.27467913, -0.02870156, 0.30729826, -1.85180733, 0.00724723, 1.82633607, 0.83968904]
cal_22_mm = [-0.27492752, -0.02881564, 0.30730066, -1.84703765, 0.00730904, 1.82137303, 0.83964258]
cal_24_mm = [-0.27495433, -0.02910731, 0.30729987, -1.84214418, 0.00736595, 1.81627148, 0.83959877]
cal_26_mm = [-0.27498185, -0.02917162, 0.30729728, -1.83711943, 0.00740111, 1.81114207, 0.83956440]
cal_28_mm = [-0.27500193, -0.02946890, 0.30729693, -1.83219466, 0.00744732, 1.80599850, 0.83953597]
cal_30_mm = [-0.27501834, -0.02951123, 0.30730128, -1.82733378, 0.00747141, 1.80099033, 0.83951795]
cal_40_mm = [-0.27535423, -0.02973620, 0.30729625, -1.80185362, 0.00748783, 1.77566828, 0.83950376]
cal_50_mm = [-0.27537779, -0.02964236, 0.30729646, -1.77550289, 0.00747043, 1.75009969, 0.83951632]
cal_60_mm = [-0.27538812, -0.02767563, 0.30729625, -1.74815756, 0.00737045, 1.72428787, 0.83968787]
cal_70_mm = [-0.27534187, -0.02516956, 0.30729125, -1.71972291, 0.00653091, 1.69838691, 0.84020591]
cal_80_mm = [-0.27489364, -0.02177300, 0.30729093, -1.69023271, 0.00559191, 1.67217843, 0.84087814]
cal_90_mm = [-0.27442663, -0.01700015, 0.30725260, -1.66094411, 0.00428391, 1.64610852, 0.84159472]
cal_100_mm = [-0.27342677, -0.01169027, 0.30724883, -1.62902436, 0.00271029, 1.61938627, 0.84278577]
cal_110_mm = [-0.27221279, -0.00529769, 0.30724584, -1.59589389, 0.00080301, 1.59226100, 0.84416299]
cal_120_mm = [-0.27068688, 0.00206569, 0.30723810, -1.56146977, -0.00141168, 1.56471790, 0.84578688]
cal_130_mm = [-0.26867866, 0.01051012, 0.30723448, -1.52532189, -0.00398714, 1.53676718, 0.84757781]
cal_140_mm = [-0.26635378, 0.02023801, 0.30722737, -1.48763902, -0.00694801, 1.50828884, 0.84963912]

# calibration 15/02/24 60DEGREE FINGERS EI1 0mm=press
cal_0_mm = [-0.26363617, -0.05473505, 0.30020690, -1.90742241, 0.00734889, 1.87581724, 0.82313380]
cal_2_mm = [-0.26366258, -0.05472722, 0.30021328, -1.90369823, 0.00735250, 1.87093787, 0.82305917]
cal_4_mm = [-0.26371654, -0.05471687, 0.30021328, -1.89936716, 0.00741290, 1.86591738, 0.82294415]
cal_6_mm = [-0.26391464, -0.05504486, 0.30021328, -1.89472156, 0.00748206, 1.86090533, 0.82281917]
cal_8_mm = [-0.26396259, -0.05532080, 0.30021161, -1.88997358, 0.00757329, 1.85586071, 0.82272108]
cal_10_mm = [-0.26403608, -0.05563658, 0.30021328, -1.88528073, 0.00770713, 1.85078742, 0.82262177]
cal_12_mm = [-0.26422203, -0.05593439, 0.30021523, -1.88056663, 0.00779554, 1.84573231, 0.82252450]
cal_14_mm = [-0.26427300, -0.05619965, 0.30021331, -1.87574604, 0.00784266, 1.84068288, 0.82243819]
cal_16_mm = [-0.26438289, -0.05632581, 0.30021690, -1.87089863, 0.00788102, 1.83561542, 0.82236489]
cal_18_mm = [-0.26453586, -0.05657995, 0.30021523, -1.86600990, 0.00791936, 1.83057723, 0.82228960]
cal_20_mm = [-0.26458381, -0.05668035, 0.30021328, -1.86109418, 0.00795603, 1.82553612, 0.82223020]
cal_22_mm = [-0.26463690, -0.05691966, 0.30021328, -1.85616508, 0.00798505, 1.82048298, 0.82216497]
cal_24_mm = [-0.26482269, -0.05695218, 0.30022009, -1.85119037, 0.00800458, 1.81538102, 0.82211627]
cal_26_mm = [-0.26486016, -0.05698160, 0.30021650, -1.84621390, 0.00801417, 1.81027087, 0.82207689]
cal_28_mm = [-0.26489700, -0.05699540, 0.30021845, -1.84108635, 0.00801764, 1.80515880, 0.82204296]
cal_30_mm = [-0.26493536, -0.05700533, 0.30021845, -1.83597784, 0.00801835, 1.80008457, 0.82202409]
cal_40_mm = [-0.26529586, -0.05696333, 0.30022013, -1.80990437, 0.00799617, 1.77460503, 0.82199937]
cal_50_mm = [-0.26544880, -0.05556902, 0.30021846, -1.78293351, 0.00772567, 1.74892897, 0.82204890]
cal_60_mm = [-0.26546751, -0.05339131, 0.30021507, -1.75493735, 0.00707560, 1.72307920, 0.82238232]
cal_70_mm = [-0.26539159, -0.05028930, 0.30021145, -1.72586789, 0.00612781, 1.69704613, 0.82291076]
cal_80_mm = [-0.26503468, -0.04624956, 0.30021437, -1.69573253, 0.00491428, 1.67075798, 0.82362430]
cal_90_mm = [-0.26424438, -0.04132709, 0.30020633, -1.66449699, 0.00342211, 1.64419101, 0.82454922]
cal_100_mm = [-0.26334276, -0.03539899, 0.30020896, -1.63179003, 0.00161609, 1.61728759, 0.82568431]

# calibration 17/02/24 45DEGREE FINGERS EI1 0mm=press, 2=touch
cal_0_mm = [-0.25846346, -0.09142749, 0.29959801, -1.92880532, 0.01334620, 1.85091865, 0.83178104]
cal_2_mm = [-0.25870198, -0.09147041, 0.29959110, -1.92550948, 0.01331499, 1.84638432, 0.83170138]
cal_4_mm = [-0.25880085, -0.09145244, 0.29958943, -1.92094128, 0.01331539, 1.84132962, 0.83144733]
cal_6_mm = [-0.25893341, -0.09144813, 0.29959277, -1.91614196, 0.01331951, 1.83633531, 0.83141448]
cal_8_mm = [-0.25902936, -0.09150171, 0.29958943, -1.91140079, 0.01332294, 1.83133743, 0.83137158]
cal_10_mm = [-0.25920749, -0.09164791, 0.29959110, -1.90661197, 0.01332703, 1.82621356, 0.83131193]
cal_12_mm = [-0.25925322, -0.09187866, 0.29958943, -1.90174872, 0.01333121, 1.82119543, 0.83124218]
cal_14_mm = [-0.25946668, -0.09193467, 0.29959277, -1.89680717, 0.01333804, 1.81617031, 0.83112259]
cal_16_mm = [-0.25952683, -0.09201289, 0.29958748, -1.89181902, 0.01333661, 1.81101491, 0.83102176]
cal_18_mm = [-0.25958494, -0.09225744, 0.29959109, -1.88687240, 0.01333972, 1.80588839, 0.83097190]
cal_20_mm = [-0.25981331, -0.09227619, 0.29959277, -1.88189885, 0.01334080, 1.80087797, 0.83093134]
cal_22_mm = [-0.25985366, -0.09229092, 0.29959472, -1.87688817, 0.01334009, 1.79581659, 0.83089471]
cal_24_mm = [-0.25992494, -0.09230065, 0.29959277, -1.87184252, 0.01334212, 1.79078809, 0.83085940]
cal_26_mm = [-0.26017999, -0.09230066, 0.29958871, -1.86667663, 0.01333933, 1.78573958, 0.83083132]
cal_28_mm = [-0.26021408, -0.09230109, 0.29959107, -1.86152746, 0.01334141, 1.78061220, 0.83080412]
cal_30_mm = [-0.26026884, -0.09230342, 0.29959277, -1.85639911, 0.01333733, 1.77549605, 0.83077798]
cal_40_mm = [-0.26082449, -0.09226030, 0.29958748, -1.82978011, 0.01331539, 1.74991513, 0.83076478]
cal_50_mm = [-0.26113845, -0.09032312, 0.29958943, -1.80233337, 0.01277609, 1.72413216, 0.83078100]
cal_60_mm = [-0.26119477, -0.08769341, 0.29958748, -1.77383102, 0.01209097, 1.69836126, 0.83088790]
cal_70_mm = [-0.26120006, -0.08401203, 0.29958589, -1.74435985, 0.01111002, 1.67226951, 0.83143025]
cal_80_mm = [-0.26115087, -0.07952169, 0.29958052, -1.71367803, 0.00987935, 1.64588992, 0.83217959]
cal_90_mm = [-0.26065998, -0.07420020, 0.29957523, -1.68188725, 0.00825537, 1.61942467, 0.83311926]
cal_100_mm = [-0.25974838, -0.06776345, 0.29957022, -1.64890949, 0.00631284, 1.59253614, 0.83427676]

# calibration 18/02/24 75DEGREE FINGERS EI2 0mm=press, 2mm=full touch
cal_0_mm = [-0.26465677, -0.06653508, 0.30426445, -1.94810009, 0.00519931, 1.90495333, 0.85446492]
cal_2_mm = [-0.26468163, -0.06687588, 0.30426973, -1.94497111, 0.00517773, 1.90039799, 0.85425492]
cal_4_mm = [-0.26471358, -0.06685380, 0.30427141, -1.94091235, 0.00518325, 1.89530001, 0.85410143]
cal_6_mm = [-0.26503110, -0.06690994, 0.30427141, -1.93651970, 0.00519738, 1.89024569, 0.85392881]
cal_8_mm = [-0.26507791, -0.06754549, 0.30427143, -1.93211447, 0.00522090, 1.88522869, 0.85378509]
cal_10_mm = [-0.26518058, -0.06801998, 0.30427503, -1.92764600, 0.00524558, 1.88022699, 0.85362153]
cal_12_mm = [-0.26540390, -0.06849060, 0.30427548, -1.92303129, 0.00527987, 1.87519219, 0.85347924]
cal_14_mm = [-0.26545875, -0.06897778, 0.30427141, -1.91841813, 0.00532508, 1.87020468, 0.85334477]
cal_16_mm = [-0.26576501, -0.06940543, 0.30427503, -1.91375831, 0.00574556, 1.86524832, 0.85320996]
cal_18_mm = [-0.26580655, -0.06978559, 0.30426974, -1.90907449, 0.00577708, 1.86022842, 0.85307874]
cal_20_mm = [-0.26585404, -0.07014882, 0.30427335, -1.90438504, 0.00580201, 1.85512281, 0.85296376]
cal_22_mm = [-0.26614276, -0.07026808, 0.30427715, -1.89974040, 0.00582059, 1.85000908, 0.85285480]
cal_24_mm = [-0.26618544, -0.07062135, 0.30427597, -1.89489858, 0.00583393, 1.84488728, 0.85274799]
cal_26_mm = [-0.26622521, -0.07069334, 0.30427338, -1.88996905, 0.00584898, 1.83986722, 0.85263867]
cal_28_mm = [-0.26626606, -0.07106799, 0.30427715, -1.88515273, 0.00586181, 1.83481416, 0.85256351]
cal_30_mm = [-0.26659076, -0.07112033, 0.30427670, -1.88033117, 0.00587356, 1.82983591, 0.85248238]
cal_40_mm = [-0.26699620, -0.07170002, 0.30427141, -1.85536463, 0.00589009, 1.80442038, 0.85213604]
cal_50_mm = [-0.26733672, -0.07169671, 0.30427670, -1.82948831, 0.00588733, 1.77903604, 0.85199639]
cal_60_mm = [-0.26760782, -0.07087460, 0.30427503, -1.80268477, 0.00586122, 1.75335092, 0.85198065]
cal_70_mm = [-0.26763740, -0.06916432, 0.30427142, -1.77486078, 0.00580894, 1.72752090, 0.85200943]
cal_80_mm = [-0.26763818, -0.06612083, 0.30426833, -1.74604985, 0.00500372, 1.70165443, 0.85220496]
cal_90_mm = [-0.26758452, -0.06232216, 0.30426923, -1.71628907, 0.00403251, 1.67534211, 0.85279979]
cal_100_mm = [-0.26716275, -0.05741868, 0.30426233, -1.68520777, 0.00271554, 1.64887705, 0.85350829]

# calibration 18/02/24 90DEGREE FINGERS EI2 0mm=press, 2mm=full touch
cal_0_mm = [-0.22735047, 0.07764017, 0.29187174, -1.80121015, -0.02719755, 1.87792673, 0.89952540]
cal_2_mm = [-0.22735047, 0.07766074, 0.29186202, -1.79806534, -0.02720234, 1.87358750, 0.89929920]
cal_4_mm = [-0.22735409, 0.07770124, 0.29185190, -1.79434772, -0.02718927, 1.86849952, 0.89923834]
cal_6_mm = [-0.22735242, 0.07762914, 0.29185190, -1.78994878, -0.02716141, 1.86361421, 0.89911325]
cal_8_mm = [-0.22735284, 0.07700316, 0.29185283, -1.78530338, -0.02711940, 1.85858461, 0.89897425]
cal_10_mm = [-0.22734769, 0.07652020, 0.29185035, -1.78081513, -0.02667632, 1.85358087, 0.89886547]
cal_12_mm = [-0.22734449, 0.07607479, 0.29184860, -1.77634064, -0.02661313, 1.84858942, 0.89873454]
cal_14_mm = [-0.22734173, 0.07565648, 0.29185035, -1.77169450, -0.02656403, 1.84356910, 0.89861984]
cal_16_mm = [-0.22733150, 0.07527275, 0.29185397, -1.76711536, -0.02651769, 1.83852584, 0.89852024]
cal_18_mm = [-0.22732211, 0.07492007, 0.29185397, -1.76240957, -0.02648662, 1.83359266, 0.89842931]
cal_20_mm = [-0.22731530, 0.07477739, 0.29185397, -1.75757535, -0.02645714, 1.82866858, 0.89834729]
cal_22_mm = [-0.22730870, 0.07447185, 0.29185397, -1.75283607, -0.02642588, 1.82373462, 0.89826242]
cal_24_mm = [-0.22727891, 0.07414268, 0.29185102, -1.74808701, -0.02584221, 1.81864916, 0.89818750]
cal_26_mm = [-0.22727540, 0.07408516, 0.29185081, -1.74334378, -0.02582160, 1.81355426, 0.89810795]
cal_28_mm = [-0.22700131, 0.07402443, 0.29184564, -1.73846758, -0.02579477, 1.80849515, 0.89803354]
cal_30_mm = [-0.22699018, 0.07370638, 0.29184926, -1.73332179, -0.02578193, 1.80342610, 0.89797816]
cal_40_mm = [-0.22664184, 0.07360134, 0.29184491, -1.70806267, -0.02574838, 1.77812006, 0.89771416]
cal_50_mm = [-0.22606942, 0.07361168, 0.29184751, -1.68172271, -0.02574944, 1.75245942, 0.89768645]
cal_60_mm = [-0.22530564, 0.07464560, 0.29184561, -1.65436262, -0.02577510, 1.72651245, 0.89768542]
cal_70_mm = [-0.22438173, 0.07737909, 0.29184758, -1.62583973, -0.02584150, 1.70047544, 0.89772618]
cal_80_mm = [-0.22337230, 0.08077094, 0.29184035, -1.59602708, -0.02623951, 1.67390127, 0.89812051]
cal_90_mm = [-0.22197642, 0.08517698, 0.29184035, -1.56507880, -0.02772294, 1.64703665, 0.89863802]
cal_100_mm = [-0.22015800, 0.09046841, 0.29183534, -1.53265202, -0.02896271, 1.61987860, 0.89934124]

speed_factor = 0.1 #0.05 # 0.1 is slow movements, 0.01 is extremely slow

test55 = [0.27176940, 0.52305102, -0.02750686, -1.05737524, 0.00430891, 1.57391222, 0.53126934]

if True: franka.move_joints(test55, speed_factor)

In [327]:
t = franka.getTransformationMatrix()
p = franka.position()
r = franka.orientation()
print("Transformation matrix is:\n", t)
print("Position is:", p)
print("Rotation is:", r)

Transformation matrix is:
 [[ 9.60108469e-01 -2.79601445e-01 -3.84182987e-03  6.79116457e-01]
 [-2.79626970e-01 -9.60049103e-01 -1.06994576e-02  1.75280336e-01]
 [-6.96761509e-04  1.13469191e-02 -9.99935379e-01  5.36857330e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Position is: [0.67911646 0.17528034 0.53685733]
Rotation is: [ 3.13089291 -0.00384184  0.28338112]


In [ ]:
camera_sequence = [
  # gripper joints: x=110mm, y=130mm, overlooking bin
  [-0.38154769, -0.23222204, 0.29172773, -1.78685710, 0.42341105, 1.52417625, 0.73060460],
  # transition to vertical orientation
  [-0.38118336, -0.23231364, 0.41147124, -1.78681521, 0.29164558, 1.56990163, 0.84120683],
  [-0.37963573, -0.25167965, 0.50942685, -1.77085599, 0.20468855, 1.57759119, 0.92477864],
  [-0.38080224, -0.25862325, 0.60248531, -1.77125124, 0.14256476, 1.57756903, 1.01133587],
  # lift up to prepare for gripper homing
  [-0.38138898, -0.25240082, 0.59158531, -1.67755285, 0.14258119, 1.50636371, 1.01062951],
  [-0.38156794, -0.21909391, 0.58025007, -1.55936750, 0.14285254, 1.40292759, 1.02373480],
  [-0.10839343, -0.13431179, 0.33850536, -1.30709421, 0.07755623, 1.17963370, 1.12086774],
  # now home
  # next move to initial position over bin
  [-0.08894655, -0.03994745, 0.47738154, -1.33025209, 0.01123709, 1.30227450, 1.20926677],
]